In [2]:
import pandas as pd
import geopandas
from shapely.geometry import Polygon, LineString, Point
from haversine import haversine, Unit
import plotly.express as px
import numpy as np

In [27]:
cam_meta = pd.read_csv("../data/cam_meta_05302022.csv")
cam_meta = cam_meta[['hpweren_camera_description', 'station', 'dir', 'cam_lat', 'cam_long']]
cam_meta = cam_meta.loc[(cam_meta['dir'] == 'n') | (cam_meta['dir'] == 'e') | (cam_meta['dir'] == 's') |(cam_meta['dir'] == 'w')].reset_index(drop=True)

In [28]:
#x=.2222
#haversine((33.1599, -116.8081), (33.1599+x, -116.8081+x), unit=Unit.MILES)

def find_fov_right(direction, lat, long, fill_dist):
    fill_dist = .2777 #25 miles away
    if direction == "n":
        return (long+fill_dist, lat+fill_dist)
    if direction == "e":
        return (long+fill_dist, lat-fill_dist)
    if direction == "s":
        return (long-fill_dist, lat-fill_dist)
    if direction == "w":
        return (long-fill_dist, lat+fill_dist)
    
def find_fov_left(direction, lat, long):
    fill_dist = .2777 #25 miles away
    if direction == "n":
        return (long-fill_dist, lat+fill_dist)
    if direction == "e":
        return (long+fill_dist, lat+fill_dist)
    if direction == "s":
        return (long+fill_dist, lat-fill_dist)
    if direction == "w":
        return (long-fill_dist, lat-fill_dist)

In [29]:
cam_meta['fov_right'] = cam_meta.apply(lambda x: find_fov_right(x['dir'], x['cam_lat'], x['cam_long']), axis=1)
cam_meta['fov_left'] = cam_meta.apply(lambda x: find_fov_left(x['dir'], x['cam_lat'], x['cam_long']), axis=1)

cam_meta
left_edges = {'n': 'w', 'e': 'n', 's':'e', 'w': 's'}
right_edges = {'n': 'e', 'e':'s', 's':'w', 'w':'n'}
intersections = []
counts = []

for i in range(len(cam_meta)):
    direction = cam_meta['dir'][i]
    intersection = []
    c1 = (cam_meta['cam_long'][i], cam_meta['cam_lat'][i])
    r1 = cam_meta['fov_right'][i]
    l1 = cam_meta['fov_left'][i]
    for j in range(len(cam_meta)):
        c2 = (cam_meta['cam_long'][j], cam_meta['cam_lat'][j])
        r2 = cam_meta['fov_right'][j]
        l2 = cam_meta['fov_left'][j]
        if geopandas.GeoSeries(Polygon([c1, r1, l1])).intersects(geopandas.GeoSeries(Polygon([c2, r2, l2])))[0] == True:
            if cam_meta['station'][j] != cam_meta['station'][i]:
                intersection.append((cam_meta['station'][j], cam_meta['dir'][j]))
    if len(intersection) > 0:
        intersections.append(intersection)
        counts.append(len(intersection))
    else:
        intersections.append(0)
        counts.append(0)    
        
cam_meta['intersections'] = intersections
cam_meta['num_intersection'] = counts

In [30]:
cam_meta.to_csv("../data/intersections.csv", index=False)

In [3]:
geopandas.GeoSeries(Polygon([c1, r1, l1]))

NameError: name 'c1' is not defined